### Problem Definition

We have smoke data we collected over time, we initially used regression to predict the probability of smoke occurence, but now we want to predict: 
- Probability of smoke detection for the given day 
- Likely time window of first smoke detection 